In [1]:
import numpy as np
import os,sys
sys.path.append(os.path.realpath('../lib'))
#module_path = os.path.abspath(os.path.join('/Users/vayer/Documents/OT/WGW/lib_wgw'))
#sys.path.append(module_path)
from custom_svc import Graph_FGW_SVC_Classifier
from data_loader import load_local_data
import time
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
X,y=load_local_data('../data','mutag',wl=2)

In [3]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.33, random_state=42)

In [4]:
fgw=Graph_FGW_SVC_Classifier(C=1,gamma=2,alpha=1e-3,method='shortest_path',
                             features_metric='hamming_dist',amijo=False,
                            verbose=True)
fgw.compute_similarity(X[2],X[2])

-1.2297855042001735e-18

In [5]:
from utils import dist
xalea = np.random.randn(10, 2)
C1 = dist(xalea, xalea)

In [6]:
#C1=X[2].C
C2=C1
p=np.ones(C1.shape[0])/C1.shape[0]
q=p

In [7]:
from FGW import init_matrix,tensor_product
from FGW import gwloss,gwggrad,gw_lp

In [8]:
constC,hC1,hC2=init_matrix(C1,C2,p,q)
tens=tensor_product(constC,hC1,hC2,p[:,None]*q[None,:])

In [9]:
tens[tens<=0]

array([], dtype=float64)

In [10]:
def f(G):
    return gwloss(constC,hC1,hC2,G)
def df(G):
    return gwggrad(constC,hC1,hC2,G)
def cost(G):
    return np.sum(M * G) + reg * f(G)

In [11]:
from ot import emd

In [12]:
M=np.zeros((C1.shape[0],C2.shape[0]))

In [13]:
alpha=1e-3

In [14]:
G=p[:,None]*q[None,:]
Mi = M + alpha * df(G) #Gradient(xt)
# set M positive
Mi += Mi.min()
# solve linear program
Gc = emd(p, q, Mi) #st

In [15]:
Gc

array([[0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0.1, 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0.1, 0. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0.1, 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0.1, 0. , 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0.1, 0. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.1, 0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.1, 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.1]])

In [16]:
tens=tensor_product(constC,hC1,hC2,Gc)

In [17]:
tens[tens<=0]

array([ 0.00000000e+00, -3.55271368e-15,  0.00000000e+00,  0.00000000e+00,
       -3.55271368e-15,  0.00000000e+00])

In [19]:
np.all(np.sum(Gc,axis=0)==p)

True

In [20]:
N=tens.shape[0]

In [21]:
tens2 = np.zeros((N,N))

In [22]:
for i in range(N):
    for j in range(N):
        for k in range(N):
            for l in range(N):
                A=(C1[i,k]-C2[j,l])**2
                tens2[i,j] += A*(Gc[k,l])

In [23]:
tens2

array([[ 0.        ,  4.75611187, 12.78956061, 28.64841719, 11.73001571,
         6.11513026,  4.9852642 , 23.91533236,  0.23430292,  5.16744751],
       [ 4.75611187,  0.        , 12.99201429, 23.42124526,  8.93978514,
        17.13793359, 15.23829735,  8.93318848,  6.69314317,  2.46162699],
       [12.78956061, 12.99201429,  0.        ,  5.40854961,  1.1674805 ,
        19.45040683, 10.54049171, 16.68477124, 13.42019763,  6.65481264],
       [28.64841719, 23.42124526,  5.40854961,  0.        ,  8.13870688,
        31.74799847, 22.56733977, 14.46483696, 29.21903757, 18.7911415 ],
       [11.73001571,  8.93978514,  1.1674805 ,  8.13870688,  0.        ,
        23.28411841, 14.22404121, 12.38084221, 13.32010223,  3.19857423],
       [ 6.11513026, 17.13793359, 19.45040683, 31.74799847, 23.28411841,
         0.        ,  1.89845332, 37.86929501,  4.0764115 , 19.27636081],
       [ 4.9852642 , 15.23829735, 10.54049171, 22.56733977, 14.22404121,
         1.89845332,  0.        , 33.49937962

In [24]:
tens

array([[ 3.55271368e-15,  4.75611187e+00,  1.27895606e+01,
         2.86484172e+01,  1.17300157e+01,  6.11513026e+00,
         4.98526420e+00,  2.39153324e+01,  2.34302918e-01,
         5.16744751e+00],
       [ 4.75611187e+00,  0.00000000e+00,  1.29920143e+01,
         2.34212453e+01,  8.93978514e+00,  1.71379336e+01,
         1.52382973e+01,  8.93318848e+00,  6.69314317e+00,
         2.46162699e+00],
       [ 1.27895606e+01,  1.29920143e+01, -3.55271368e-15,
         5.40854961e+00,  1.16748050e+00,  1.94504068e+01,
         1.05404917e+01,  1.66847712e+01,  1.34201976e+01,
         6.65481264e+00],
       [ 2.86484172e+01,  2.34212453e+01,  5.40854961e+00,
         7.10542736e-15,  8.13870688e+00,  3.17479985e+01,
         2.25673398e+01,  1.44648370e+01,  2.92190376e+01,
         1.87911415e+01],
       [ 1.17300157e+01,  8.93978514e+00,  1.16748050e+00,
         8.13870688e+00,  0.00000000e+00,  2.32841184e+01,
         1.42240412e+01,  1.23808422e+01,  1.33201022e+01,
         3.

In [26]:
C2

array([[ 0.        ,  1.32056891,  3.96342705,  7.20931476,  3.30554976,
         1.18665914,  1.63077163,  5.12796426,  0.06423588,  1.2901006 ],
       [ 1.32056891,  0.        ,  4.82220966,  7.34424077,  3.00112768,
         4.97043012,  5.18630458,  1.6610644 ,  1.96215166,  0.48716071],
       [ 3.96342705,  4.82220966,  0.        ,  0.52733147,  0.3495778 ,
         4.80822757,  2.70602923,  4.73575066,  3.99529836,  2.26525122],
       [ 7.20931476,  7.34424077,  0.52733147,  0.        ,  0.95811357,
         8.47399541,  5.48124319,  5.50715951,  7.35538004,  4.06461455],
       [ 3.30554976,  3.00112768,  0.3495778 ,  0.95811357,  0.        ,
         5.35440846,  3.53848535,  2.5123832 ,  3.61226782,  1.07600423],
       [ 1.18665914,  4.97043012,  4.80822757,  8.47399541,  5.35440846,
         0.        ,  0.41019151, 10.43311416,  0.69931871,  4.19953479],
       [ 1.63077163,  5.18630458,  2.70602923,  5.48124319,  3.53848535,
         0.41019151,  0.        ,  9.33951532

In [ ]:
G,log=gw_lp(C1,C2,p,q,'square_loss',amijo=True,log=True)

In [ ]:
log['loss'][::-1][0]

In [ ]:
d = np.zeros((5,2))

In [ ]:
def f(G):
    return gwloss(constC,hC1,hC2,G)
def df(G):
    return gwggrad(constC,hC1,hC2,G)

In [ ]:
T=np.outer(p,q)

In [ ]:
Mi = M + reg * df(G) #Gradient(xt)
# set M positive
Mi += Mi.min()

# solve linear program
Gc = emd(p, q, Mi) #st

In [ ]:
constCprime,hC1,hC2=init_matrix(C1,C2,G+alpha*deltaG,p,q)

In [ ]:
sim1=[]
sim2=[]
no_amijo_time=[]
amijo_time=[]
N=500
for i in range(N):
    n1=np.random.randint(0,X.shape[0])
    n2=np.random.randint(0,X.shape[0])
    st=time.time()
    sim1.append(Graph_FGW_SVC_Classifier(C=1,gamma=2,alpha=1e-3,method='shortest_path',features_metric='hamming_dist',
                                         amijo=False).compute_similarity(X[n1],X[n2]))
    ed=time.time()
    no_amijo_time.append(ed-st)
    #clf.set_params(amijo=True)
    st=time.time()
    sim2.append(Graph_FGW_SVC_Classifier(C=1,gamma=2,alpha=1e-3,method='shortest_path',features_metric='hamming_dist',
                                         amijo=True).compute_similarity(X[n1],X[n2]))
    ed=time.time()
    amijo_time.append(ed-st)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(sim1)
plt.plot(sim2)
plt.legend(['no_amijo','amijo'])
plt.show()

In [ ]:
np.max(np.abs(np.array(sim2)-np.array(sim1)))

In [ ]:
np.all(np.array(sim1)>0)

In [ ]:
np.array(sim1)[np.array(sim1)<0]

In [ ]:
fgw=Graph_FGW_SVC_Classifier(C=1,gamma=2,alpha=1e-3,method='shortest_path',features_metric='hamming_dist',amijo=False)
fgw.compute_similarity(X_train[124],X_train[124])

In [ ]:
X_train[124].nodes()

In [ ]:
graph_svc=Graph_FGW_SVC_Classifier(C=1,gamma=2,alpha=1e-3,method='shortest_path',features_metric='hamming_dist',wl=2)

In [ ]:
graph_svc.compute_all_distance(X_train,X_train)

In [ ]:
graph_svc.D[graph_svc.D<0]